In [1]:
from langchain_community.document_loaders import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
import os
from pymongo import MongoClient
from langchain.vectorstores import MongoDBAtlasVectorSearch

In [110]:
loader = CSVLoader("cleaned_data.csv")
data = loader.load()

In [111]:
type(data)

list

In [112]:
print(data[10].page_content)

NamaProduk: ZO14 Kain Lap Dapur Kawat Bentuk Petak Halus Untuk Membersih Panci
HargaProduk: 638
LokasiToko: Jakarta Pusat
LinkProduk: https://www.tokopedia.com/autotrader/zo14-kain-lap-dapur-kawat-bentuk-petak-halus-untuk-membersih-panci?extParam=ivf%3Dfalse


In [113]:
text_spliter = RecursiveCharacterTextSplitter(chunk_size=350, chunk_overlap=0,
                                               separators=["\n\n", "\n", " "],
                                             length_function=len)

In [114]:
text_chunk = text_spliter.split_documents(data)

In [115]:
load_dotenv('.env')

True

In [116]:
KEY=os.getenv("OPENAI_API_KEY")
embedding = OpenAIEmbeddings(openai_api_key=KEY)


In [117]:
mongo_uri = os.getenv("MONGODB_URI")
client = MongoClient(mongo_uri)

In [118]:
collection = client['search_db']['search_col']

In [119]:
# Reset w/out deleting the Search Index 
collection.delete_many({})

DeleteResult({'n': 1131, 'electionId': ObjectId('7fffffff0000000000000396'), 'opTime': {'ts': Timestamp(1729602023, 1226), 't': 918}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1729602023, 1227), 'signature': {'hash': b'$\x8c`m\\\xf2\xa8\xcb\x01\xc2bI\xfa\x19\xf8LV\xb7\xda|', 'keyId': 7367306751726583810}}, 'operationTime': Timestamp(1729602023, 1226)}, acknowledged=True)

In [120]:
docsearch = MongoDBAtlasVectorSearch.from_documents(
    text_chunk, embedding, collection=collection, index_name="vsearch_index"
)